In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Covid-19 Advanced Analysis in India and Predictions using Fb-Prophet

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Covid_Testing = pd.read_csv('/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv')
Covid_Testing.head()

In [ ]:
Covid_Testing['State'].unique()

In [ ]:
print(Covid_Testing['Date'].min())
print(Covid_Testing['Date'].max())

In [ ]:
Covid_Summary = pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
Covid_Summary.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
Covid_Summary.isnull().sum()

In [ ]:
Covid_Summary.info()

In [ ]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

In [ ]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.tail()

In [ ]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'])

In [ ]:
Covid_Summary.head()

In [ ]:
Covid_Summary.info()

In [ ]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [ ]:
india_cases.head()

In [ ]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

In [ ]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()

In [ ]:
Trend.head()

In [ ]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [ ]:
import plotly.express as px

In [ ]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index,
                     'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Orange')

In [ ]:
top_10_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
            'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','blue')

In [ ]:
top_10_recovered_states = india_cases.sort_values('Cured', ascending=False)[:10]
horizontal_bar_chart(top_10_recovered_states, top_10_recovered_states.Cured, top_10_recovered_states.index,
                    'Top 10 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Purple')

In [ ]:
vaccination = pd.read_csv(r"../input/covid19-in-india/covid_vaccine_statewise.csv")

In [ ]:
vaccination.tail()

In [ ]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [ ]:
Maharashtra = vaccination[vaccination["State"]=="Maharashtra"]
fig = px.line(Maharashtra,x="Date",y="Total Vaccinatons",title="Vaccination till date in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True) 

In [ ]:
fig = px.line(Maharashtra,x="Date",y="Total CoviShield Administered",title="CoviShield Administered in Mahrashtra")  
fig.update_xaxes(rangeslider_visible=True) 

In [ ]:
fig = px.line(Maharashtra,x="Date",y="Total Covaxin Administered",title="Covaxin Administered in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True) 

For the prediction purpose we have use Prophet library produced by Facebook which is used for Time series Forecasting. Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data.

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode

In [ ]:
model = Prophet()

In [ ]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

In [ ]:
Cured = Trend.loc[:, ['Date', 'Cured']] 
Cured.tail()

In [ ]:
# Dataframe must have columns "Date" and "Values" with the dates and values  for prophet prediction
Confirmed.columns = ['ds', 'y']
model.fit(Confirmed)

In [ ]:
future = model.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future.tail()

In [ ]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_conf = model.predict(future)
forecast_india_conf

In [ ]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [ ]:
model1 = Prophet()

In [ ]:
# Dataframe must have columns "Date" and "Values" with the dates and values  for prophet prediction
Cured.columns = ['ds', 'y']
model1.fit(Cured)

future_1 = model1.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future_1.tail()

In [ ]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_cured = model1.predict(future_1)
forecast_india_cured

In [ ]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig) 